Import data in PNG form and split into Train, Test, Validate and load


In [3]:
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split
from torchvision.transforms import ToTensor, Normalize, Compose, Grayscale

train_set = '../Data/mnist-png-format/train'
test_set = '../Data/mnist-png-format/test'
 
batch_size = 64
workers = 4

transforms = Compose([
                        Grayscale(num_output_channels=1),   # PNG file is RGB 3 layer convert to greyscal 1 layer
                        ToTensor(),           
                        Normalize(mean=(0.5), std=(0.5)), 
                        ])

train_dataset = ImageFolder(train_set, transform=transforms)
test_dataset = ImageFolder(test_set, transform=transforms)

# Split Train dataset set into two (Train=85/Validation=15% split)
train_size = int(len(train_dataset) * 0.85) 
validation_size = (len(train_dataset) - train_size) 
train, validation = random_split(train_dataset, [train_size, validation_size])

# load data into usable format, mix/shuffle data so data is not in order 
train_data = DataLoader(train, batch_size = batch_size, shuffle = True, num_workers = workers)
val_data  = DataLoader(validation, batch_size = batch_size, shuffle = True, num_workers = workers)
test_data  = DataLoader(test_dataset, batch_size = batch_size, shuffle = True, num_workers = workers)

In [4]:
from torch import nn, optim
import torch

class SingleLayerModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden_1 = nn.Linear(28*28,100) # 28x28 Input image, number of neurons
        self.output = nn.Linear(100,10) # 10 class output 0-9


    def forward(self,x):
        x = x.view(x.shape[0], -1) 
        x = nn.functional.relu(self.hidden_1(x))
        y = self.output(x)
        return y

In [7]:
ModelMLP  = SingleLayerModel()
optimiser = optim.Adam(ModelMLP.parameters())
loss_func = nn.CrossEntropyLoss()
train_epochs = 3

In [8]:
def evaluate_model(model, eval_dataset):
    model.eval() # Set model mode to evaluation not training
    correct_batch = 0

    with torch.no_grad():                 # Gradient graph not require for evaluation should reduce memory usage
      for (imgs, labels) in eval_dataset: # loop through each batch  
        pred_y = model(imgs)
        loss = loss_func(pred_y, labels)

        pred = torch.max(pred_y, 1)[1]
        correct_batch += (pred == labels).sum().item() # Calculate correct prediction where prediction == label 
    return correct_batch / len(eval_dataset.dataset), loss.item()

In [6]:
  for i in range(train_epochs): # training epochs
    correct_batch = 0

    for (imgs, labels) in train_data: # loop through each batch  
      ModelMLP.train()
      pred_y = ModelMLP(imgs)
      loss = loss_func(pred_y, labels)

      optimiser.zero_grad()
      loss.backward()
      optimiser.step()

      pred = torch.max(pred_y, 1)[1]
      correct_batch += (pred == labels).sum().item() # Calculate correct prediction where prediction == label 

    val_acc, val_loss = evaluate_model(ModelMLP, val_data)

    print('Train accuracy = {0:.2%}, Train loss = {1:.3f} | \
           Validation accuracy = {2:.2%}, Validation loss = {3:.3f}'.format(correct_batch / len(train_data.dataset), loss.item(), val_acc, val_loss ))


NameError: name 'train_epochs' is not defined

In [ ]:
test, _ = evaluate_model(ModelMLP, test_data)
print('Test accuracy = {0:.2%}'.format(test))